In [ ]:
! pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 38.3 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
# for reading japanese
!pip install spacy[ja]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 3.0 MB/s eta 0:00:00


In [ ]:
# libraries for text reading
import fitz
from tqdm import  tqdm
import pandas as pd
import numpy as np
from spacy.lang.en import English
import random
import re
import torch
from sentence_transformers import SentenceTransformer, util
from time import perf_counter as timer


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# file_path = '/content/Convolutional_Neural_Network.pdf'
file_path = '/content/Neural_Network (1).pdf'
# file_path = '/content/Speed_Master_N3-Choukai-P.3.pdf'

In [ ]:
def text_formatter(text: str) -> str:
  cleaned_text = text.replace('\n', ' ').strip()

  return cleaned_text

def open_and_read_pdf(pdf_path: str)->list[dict]:
  doc = fitz.open(pdf_path)
  pages_and_texts = []
  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text)
    pages_and_texts.append({'page_number': page_number,
                           'page_char_count': len(text),
                           'page_word_count': len(text.split(' ')),
                           'page_sentence_count_raw': len(text.split('.')),
                           'page_token_count': len(text) / 4,
                           'text': text})
  return pages_and_texts


In [ ]:
pages_and_texts = open_and_read_pdf(file_path)
pages_and_texts[:2]

FileNotFoundError: no such file: '/content/Neural_Network (1).pdf'

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.head()

In [ ]:
df.describe()

In [ ]:
# nlp = English()
nlp = English()

# add a sentencizer pipeline
nlp.add_pipe('sentencizer')

# Example
doc = nlp('This is an example sentence. This is second sentence. This is the last sentence.')
assert len(list(doc.sents)) == 3

list(doc.sents)

In [ ]:
for item in tqdm(pages_and_texts):
  item['sentences'] = list(nlp(item['text']).sents)
  # change all item in sentences to string
  item['sentences'] = [str(sentence) for sentence in item['sentences']]
  # count the sentences

  item['pages_sentence_count_spacy'] = len(item['sentences'])


In [ ]:
random.sample(pages_and_texts, k=1)

In [ ]:
df = pd.DataFrame(pages_and_texts)
df

In [ ]:
# chunking our sentences together
num_chunk_size = 30
def split_list(input_list:list[str],
               slice_size: int=num_chunk_size) -> list[list[str]]:
  return [input_list[i: i + slice_size] for i in range(0, len(input_list), slice_size)]



In [ ]:
print(split_list(df.loc[1,'sentences'], num_chunk_size))

In [ ]:
# split sentences into chunks
for item in tqdm(pages_and_texts):
  item['sentences_chunks'] = split_list(item['sentences'],
                                        num_chunk_size)
  item['num_chunks'] = len(item['sentences_chunks'])

In [ ]:
random.sample(pages_and_texts, k = 2)

In [ ]:
# split each chunk into it own  item

page_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item['sentences_chunks']:
        chunk_dict = {}
        chunk_dict['page_number'] = item['page_number']


        joined_sentences_chunk = ''.join(sentence_chunk).replace('  ', ' ').strip()
        joined_sentences_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentences_chunk) # add space before capitalize words

        chunk_dict['sentence_chunk'] = joined_sentences_chunk
        # stats of chunk
        chunk_dict['chunk_char_count'] = len(joined_sentences_chunk)
        chunk_dict['chunk_word_count'] = len([word for word in joined_sentences_chunk.split(' ')])
        chunk_dict['chunk_token_count'] = len(joined_sentences_chunk) / 4

        page_and_chunks.append(chunk_dict)



In [ ]:
random.sample(page_and_chunks, k = 1)

In [ ]:
df = pd.DataFrame(page_and_chunks)
df

In [ ]:
# minimize chunks
min_token_length = 10
for row in df[df['chunk_token_count']<= min_token_length].sample(1, replace = True).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} , Text: {row[1]["sentence_chunk"]}')


In [ ]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

# Sentence embedding


In [ ]:
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device="cpu") # choose the device to load the model to (note: GPU will often be *much* faster than CPU)

# Create a list of sentences to turn into numbers
sentences = [
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]

# Sentences are encoded/embedded by calling model.encode()
embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

# See the embeddings
# for sentence, embedding in embeddings_dict.items():
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
embedding_model.to(device)

for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])


In [ ]:
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]


In [ ]:
# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32,
                                               convert_to_tensor=True) # optional to return embeddings as tensor instead of array

text_chunk_embeddings

In [ ]:
embedding_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embedding_df.head()

# RAG

In [ ]:

# Convert texts and embedding df to list of dicts
pages_and_chunks = embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings

# Query

In [ ]:
# Define helper function to print wrapped text
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [ ]:
query = input('Input something to find: ')
print(f"Query: {query}")

# 2. Embed the query to the same numerical space as the text examples
query_embedding = embedding_model.encode(query, convert_to_tensor=True)
# 3. Get similarity scores with the dot product (we'll time this for fun)
start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# 4. Get the top-k results ( 5)
top_results_dot_product = torch.topk(dot_scores, k=3)
top_results_dot_product

print("Results:")
# Loop through zipped together scores and indicies from torch.topk
for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
    print(f"Score: {score:.4f}")
    # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
    print("Text:")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    # Print the page number too so we can reference the textbook further (and check the results)
    print(f"Page number: {pages_and_chunks[idx]['page_number']}")
    print("\n")

# Extract Images from PDF

In [ ]:
!pip install PyPDF2

In [ ]:
from PyPDF2 import PdfReader

In [ ]:
# get all image from the document
reader = PdfReader(file_path)
image_paths = []
for i in range(len(reader.pages)):
    page = reader.pages[i]
    count = 0

    for image_file_object in page.images:
        with open(str(count) + image_file_object.name, "wb") as fp:
            fp.write(image_file_object.data)
            image_paths.append(str(count) + image_file_object.name)
            count += 1


In [ ]:
image_paths

# Image captioning


Link HuggingFace: https://huggingface.co/OpenGVLab/InternVL2-Llama3-76B

--Requires GPU--

In [ ]:
# import library for image captioning
from transformers import AutoProcessor
from transformers import AutoModelForCausalLM
import fitz
import io
from PIL import Image
import requests

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
checkpoint = "microsoft/git-base"
processor = AutoProcessor.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/707M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
model.to(device)

GitForCausalLM(
  (git): GitModel(
    (embeddings): GitEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (image_encoder): GitVisionModel(
      (vision_model): GitVisionTransformer(
        (embeddings): GitVisionEmbeddings(
          (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
          (position_embedding): Embedding(197, 768)
        )
        (pre_layrnorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (encoder): GitVisionEncoder(
          (layers): ModuleList(
            (0-11): 12 x GitVisionEncoderLayer(
              (self_attn): GitVisionAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
                (v_proj): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
image_paths = ['/content/451211249_1017171169542650_372178963082746065_n.png', '/content/451299471_2173929109673080_5667175673889056003_n.png']

In [ ]:
for path in image_paths:
    image = Image.open(path).convert('RGB')
    inputs = processor(images=image, return_tensors="pt").to(device)
    pixel_values = inputs.pixel_values
    generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(f'Image: {path}:')
    print(generated_caption)

Image: /content/451211249_1017171169542650_372178963082746065_n.png:
the mechanism of the mechanical device.
Image: /content/451299471_2173929109673080_5667175673889056003_n.png:
the mechanism of the gears


## ORC (testing)

In [ ]:
import warnings, logging
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)
from IPython.display import Image
from transformers import pipeline

In [ ]:
! pip install transformers -q

In [ ]:
ocr = pipeline('image-to-text', model = "microsoft/trocr-base-handwritten")

In [ ]:
Image(filename='/content/451211249_1017171169542650_372178963082746065_n.png')

In [ ]:
ocr('/content/451211249_1017171169542650_372178963082746065_n.png')